In [1]:
queriesData = []
queryStreaks = {}
banned = []
bannedKeywords = ['link:', '-host:', 'url:', 'like:']

In [2]:
def printData(data):
    for item in data:
        print(item['id'])
        
def updateData(data, query, time):
    i = 0
    exists = False
    for item in data:
        if(item['id'] == query['id']):
            exists = True
            if(time - item['timestamp'] < 30):
                countStreak(item)
                countStreak(query)
            break
        i += 1
    if(exists):
        data[i] = query
    else:
        data.append(query)
    return data

def convertTimestamp(timestamp):
    temp = timestamp[11:]
    temp = temp.replace(":", "")
    return int(temp)
            
def cleanData(data, time):
    try:
        i = 0
        for item in data:
            if(time - item['timestamp'] > 30):
                del data[i]
                i -= 1
            i += 1
    except IndexError as ex:
        print(ex)
    return data

def countStreak(query):
    streak = {}
    streak['text'] = query['text']
    streak['timestamp'] = query['timestamp']
    isBanned = True
    id = query['id']
    if(id not in banned):
        isBanned = False
    if(not isBanned):
        if(id in queryStreaks):
            queryStreaks[id].append(streak)
        else:
            queryStreaks[id] = [streak]
            
def buildChains(queryStreakData):
    tempList = []
    tempDict = {}
    
    currentTime = -1
    for item in queryStreakData:
        for query in queryStreakData[item]:
            if(currentTime == -1):
                currentTime = query['timestamp']
            if(abs(query['timestamp'] - currentTime) < 30):
                tempList.append(query['text'])
            else:
                if(item in tempDict):
                    tempDict[item].append(tempList)
                else:
                    tempDict[item] = [tempList]
                tempList = []
                tempList.append(query['text'])
                currentTime = query['timestamp']
    return tempDict


In [3]:
import codecs
file = codecs.open('data.txt', 'r','utf-8')
i = 0

for line in file:
    i += 1
    fields = line.split('\t')
    try:
        id = fields[0]
        text = fields[1]
        timestamp = fields[2]
        if any(word in text for word in bannedKeywords):
            banned.append(id)
        else:
            query = {}
            query['id'] = id
            query['text'] = text
            query['date'] = timestamp
            query['timestamp'] = convertTimestamp(timestamp)
            currentTime = query['timestamp']
            queriesData = updateData(queriesData, query, currentTime)
        if(i % 500 == 0):
            queriesData = cleanData(queriesData, currentTime)
        if( i > 100000):
            break
    except IndexError as er:
        print(i)
file.close()


In [4]:
queryStreaks = buildChains(queryStreaks)

In [6]:
print(len(queryStreaks))

3262


In [9]:
import json
def export():
    with open('streaks.json', 'w') as outfile:  
        json.dump(queryStreaks, outfile)

In [10]:
export()